In [1]:
import numpy as np

**Mendoza Hernandez Carlos Emiliano**

# CART

La idea basica de este algoritmo es dividir repetidamente los registros disponibles buscando maximizar la *homogeneidad* en los conjuntos obtenidos en dicha division.

Es un modelo jerarquico de aprendizaje que puede ser usado tanto para clasificacion como para regresion: permiten explorar relaciones complejas entre entradas y salidas sin necesidad de hacer suposiciones sobre los datos. Los arboles de decision pueden verse como una funcion $f$ que realiza una estimacion de un *mapeo* desde un espacio de entrada $X$ hacia un espacio objetivo $y: y=f(X)$. Para el caso de la regresion, el espacio objetivo $y$ es numerico: $y \in \mathbb{R}$; para la clasificacion, los valores son categoricos: $y \in \{ C_1, C_2,... \}$, se trata de un modelo de aprendizaje supervisado; por tanto, se tienen muestras con la salida esperada. La representacion mas comun para CART es un arbol binario.

Un arbol de decision divide el espacio $X$ en regiones locales utilizando alguna medida de distancia y el objetivo es determinar particiones bien separadas y homogeneas. Las regiones se dividen de acuerdo a preguntas de prueba y, con esto, se obtiene una division $n$-aria, de forma que mientras se recorre el arbol, en cada nodo se toman decisiones.

--------------

**Algorithm 1.1** Obtencion de particiones

--------------

Elegir una variable $x_i$

+ Elegir algun valor $s_i$ para $x_i$ que divida los datos de entrenamiento en dos particiones (no necesariamente iguales)
+ Medir la *pureza* (homogeneidad) resultante en cada particion
+ Usar otro valor $s_j$ para $x_i$ buscando incrementar la pureza de las particiones hasta alcanzar el umbral de *pureza aceptable*

Repetir el proceso de particion con una variable diferente (posiblemente alguna usada previamente) cada valor obtenido se convierte en un nodo en el arbol.

Implementar desde cero el algoritmo *Classification and Regrerssion Trees* (CART), incluyendo al menos las métricas de pureza:
1. *Estimate of Positive Correctness*
2. *Variance reduction*

### Grado de impureza de Gini

El indice de impureza en un rectangulo $A$ que contiene $m$ clases, se calcula como:

$$I(A) = 1-\sum_{i=1}^{m} p_i^2$$

Con $p$ la proporcion de casos en el rectangulo $A$ que pertenecen a la clase $i$.

### Grado de entropia

El grado de entropia en un area $A$ que contiene $m$ clases, se calcula como:

$$E(A) = \sum_{i=1}^{m} p_i \times \log_2 (p_i)$$

Con $p$ la proporcion de casos en $A$ que pertenecen a la clase $i$.

### Estimate of postive correctness

$$EPC = \frac{|L|}{|D|} \cdot p_L + \frac{|R|}{|D|} \cdot p_R$$

### Variance reduction

$$VR = Var(D)-\left( \frac{|L|}{|D|} \cdot Var(L) + \frac{|R|}{|D|} \cdot Var(R) \right)$$

In [2]:
class CART:
    def __init__(self, max_depth=None, min_samples_split=2, metric='gini'):
        self.tree = None
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.metric = metric

    def gini_impurity(self, y, left, right):
        def gini(idx):
            if len(idx) == 0:
                return 0
            _, counts = np.unique(y[idx], return_counts=True)
            p = counts / len(idx)
            return 1 - np.sum(p**2)
        left_gini = gini(left)
        right_gini = gini(right)
        total = len(left) + len(right)
        weighted_gini = (len(left) / total) * left_gini + (len(right) / total) * right_gini
        return 1 - weighted_gini
    
    def entropy_degree(self, y, left, right):
        def entropy(idx):
            if len(idx) == 0:
                return 0
            _, counts = np.unique(y[idx], return_counts=True)
            p = counts / len(idx)
            return -np.sum(p * np.log2(p))
        left_entropy = entropy(left)
        right_entropy = entropy(right)
        total = len(left) + len(right)
        weighted_entropy = (len(left) / total) * left_entropy + (len(right) / total) * right_entropy
        return 1 - weighted_entropy
    
    def estimate_of_positive_correctness(self, y, left, right):
        def epc(idx):
            if len(idx) == 0:
                return 0
            _, counts = np.unique(y[idx], return_counts=True)
            return counts.max() / len(idx)
        left_epc = epc(left)
        right_epc = epc(right)
        return (len(left) * left_epc + len(right) * right_epc) / len(y)
    
    def variance_reduction(self, y, left, right):
        def variance(idx):
            if len(idx) == 0:
                return 0
            return np.var(y[idx])
        left_variance = variance(left)
        right_variance = variance(right)
        weighted_variance = (len(left) / len(y)) * left_variance + (len(right) / len(y)) * right_variance
        return np.var(y) - weighted_variance
    
    def find_best_split(self, X, y):
        best_split = None
        best_score = -np.inf
        _, num_features = X.shape

        for feature in range(num_features):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left = np.where(X[:, feature] <= threshold)[0]
                right = np.where(X[:, feature] > threshold)[0]
                if len(left) < self.min_samples_split or len(right) < self.min_samples_split:
                    continue

                if self.task == 'classification':
                    if self.metric == 'gini':
                        score = self.gini_impurity(y, left, right)
                    elif self.metric == 'entropy':
                        score = self.entropy_degree(y, left, right)
                    elif self.metric == 'epc':
                        score = self.estimate_of_positive_correctness(y, left, right)
                elif self.task == 'regression':
                    score = self.variance_reduction(y, left, right)

                if score > best_score:
                    best_score = score
                    best_split = {"feature": feature, "threshold": threshold, "indices": (left, right)}
        return best_split
    
    def create_leaf(self, y):
        if self.task == 'classification':
            return np.argmax(np.bincount(y))
        elif self.task == 'regression':
            return np.mean(y)
        
    def build_tree(self, X, y, depth=0):
        num_samples, _ = X.shape
        if num_samples < self.min_samples_split or  depth >= self.max_depth:
            return self.create_leaf(y)
        
        best_split = self.find_best_split(X, y)
        if best_split is None:
            return self.create_leaf(y)
        
        left, right = best_split["indices"]
        left_tree = self.build_tree(X[left], y[left], depth+1)
        right_tree = self.build_tree(X[right], y[right], depth+1)
        return {"feature": best_split["feature"], "threshold": best_split["threshold"], "left": left_tree, "right": right_tree}
    
    def traverse_tree(self, x, tree):
        if not isinstance(tree, dict):
            return tree
        if x[tree["feature"]] <= tree["threshold"]:
            return self.traverse_tree(x, tree["left"])
        else:
            return self.traverse_tree(x, tree["right"])
        
    def fit(self, X, y, task='classification'):
        self.task = task
        self.tree = self.build_tree(X, y)
    
    def predict(self, X):
        return np.array([self.traverse_tree(x, self.tree) for x in X])

In [3]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
X, y = make_classification(n_samples=1000, n_features=5, n_classes=2, n_informative=3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [4]:
cart_model = CART(min_samples_split=10, max_depth=5, metric='gini')
cart_model.fit(X_train, y_train, task='classification')
y_pred = cart_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Resultados con métrica Gini:")
print(f"Accuracy:  {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall:    {recall:.2f}")
print(f"F1 Score:  {f1:.2f}")

Resultados con métrica Gini:
Accuracy:  0.94
Precision: 0.95
Recall:    0.94
F1 Score:  0.94


In [5]:
cart_model_entropy = CART(min_samples_split=10, max_depth=5, metric='entropy')
cart_model_entropy.fit(X_train, y_train, task='classification')
y_pred_entropy = cart_model_entropy.predict(X_test)

accuracy_entropy = accuracy_score(y_test, y_pred_entropy)
precision_entropy = precision_score(y_test, y_pred_entropy)
recall_entropy = recall_score(y_test, y_pred_entropy)
f1_entropy = f1_score(y_test, y_pred_entropy)

print("\nResultados con métrica Entropía:")
print(f"Accuracy:  {accuracy_entropy:.2f}")
print(f"Precision: {precision_entropy:.2f}")
print(f"Recall:    {recall_entropy:.2f}")
print(f"F1 Score:  {f1_entropy:.2f}")


Resultados con métrica Entropía:
Accuracy:  0.93
Precision: 0.96
Recall:    0.91
F1 Score:  0.93


In [6]:
cart_model_epc = CART(min_samples_split=10, max_depth=5, metric='epc')
cart_model_epc.fit(X_train, y_train, task='classification')
y_pred_epc = cart_model_epc.predict(X_test)

accuracy_epc = accuracy_score(y_test, y_pred_epc)
precision_epc = precision_score(y_test, y_pred_epc)
recall_epc = recall_score(y_test, y_pred_epc)
f1_epc = f1_score(y_test, y_pred_epc)

print("\nResultados con métrica EPC:")
print(f"Accuracy:  {accuracy_epc:.2f}")
print(f"Precision: {precision_epc:.2f}")
print(f"Recall:    {recall_epc:.2f}")
print(f"F1 Score:  {f1_epc:.2f}")


Resultados con métrica EPC:
Accuracy:  0.92
Precision: 0.93
Recall:    0.90
F1 Score:  0.91


In [11]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
X, y = make_regression(n_samples=1000, n_features=5, noise=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

cart_model = CART(min_samples_split=5, max_depth=5, metric='variance')
cart_model.fit(X_train, y_train, task='regression')

y_pred = cart_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("Resultados de regresión:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")


Resultados de regresión:
Mean Squared Error (MSE): 1507.25
Mean Absolute Error (MAE): 30.79
